# Clustering Client

In [1]:
from dask.distributed import Client

client = Client("tcp://127.0.0.1:61223")
client

Client Scheduler: tcp://127.0.0.1:61223 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 4 Cores: 16 Memory: 34.27 GB


In [2]:
import dask.array as da
import dask.dataframe as dd
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
from sklearn.cluster import AgglomerativeClustering
from sklearn.cluster import DBSCAN
from sklearn.metrics import silhouette_score
from dask_ml.decomposition import PCA
from sklearn import decomposition
from sklearn import manifold
from dask_ml import preprocessing

import seaborn as sns
import matplotlib.pyplot as plt
sns.set_style("whitegrid")
%matplotlib inline

## 1 - Import des jeux de données

In [3]:
data_3_months = dd.read_csv("data/clean/data_3_months.csv", parse_dates=["order_purchase_timestamp"])
data_6_months = dd.read_csv("data/clean/data_6_months.csv", parse_dates=["order_purchase_timestamp"])
data_9_months = dd.read_csv("data/clean/data_9_months.csv", parse_dates=["order_purchase_timestamp"])
data_12_months = dd.read_csv("data/clean/data_12_months.csv", parse_dates=["order_purchase_timestamp"])
data_all = dd.read_csv("data/clean/data_all.csv", parse_dates=["order_purchase_timestamp"])

## 2 - Features engineering

In [4]:
#Done
def review(df):
    grouped = df.groupby("customer_unique_id").agg({"review_score": ["count", "min", "max", "mean"]})
    grouped.columns = ["_".join(x) for x in grouped.columns.ravel()]
    return grouped

#Done
def value(df):
    grouped = df.groupby("customer_unique_id").agg({
        "payment_value": ["mean", "min", "max"]})
    grouped.columns = ["_".join(x) for x in grouped.columns.ravel()]
    return grouped

#Done
def payment_facility(df):
    df_pf = df.groupby("customer_unique_id").agg({
        "payment_installments": "sum"
    })
    cond_mask = (df_pf["payment_installments"] > 1)

    df_pf["payment_installments"] = df_pf["payment_installments"].mask(cond=cond_mask, other=0)
    df_pf = df_pf.rename(columns={"payment_installments": "payment_facility"})
    return df_pf

#Done
def category_transf(df):
    df_ct = dd.get_dummies(df[["product_category_name", "customer_unique_id"]].categorize(), columns=['product_category_name'], prefix="PCat").groupby(['customer_unique_id']).sum()
    return df_ct

#Done
def state(df):
    df_state = dd.get_dummies(df[["customer_state", "customer_unique_id"]].categorize(), columns=["customer_state"], prefix="CState").groupby(["customer_unique_id"]).sum()
    return df_state
#Done
def payment(df):
    df_p = dd.get_dummies(df[["payment_type", "customer_unique_id"]].categorize(), columns=['payment_type'], prefix="PType").groupby(['customer_unique_id']).sum()
    return df_p

# Panda
def rfm(df):
    df = df.compute()
    return df.groupby("customer_unique_id").agg(
        recency=("order_purchase_timestamp", lambda date: (df["order_purchase_timestamp"].max() - date.max()).days),
        frequency=("order_id", "count"),
        monetary=("payment_value", sum))

def features_engineering(df):
    df_fe = dd.merge(rfm(df), value(df))
    df_fe = dd.merge(df_fe, review(df))
    df_fe = dd.merge(df_fe, payment(df))
    df_fe = dd.merge(df_fe, state(df))
    df_fe = dd.merge(df_fe, category_transf(df))
    df_fe = dd.merge(df_fe, payment_facility(df))
    return df_fe

In [5]:
data_3_months_fe = features_engineering(data_3_months)
data_3_months_fe.head()

,recency,frequency,monetary,payment_value_mean,payment_value_min,payment_value_max,review_score_count,review_score_min,review_score_max,review_score_mean,...,PCat_fashion_underwear_beach,PCat_books_technical,PCat_dvds_blu_ray,PCat_security_and_services,PCat_tablets_printing_image,PCat_furniture_bedroom,PCat_cine_photo,PCat_fashio_female_clothing,PCat_books_imported,payment_facility
customer_unique_id,,,,,,,,,,,,,,,,,,,,,
0000f46a3911fa3c0805444483337064,20,1,86.22,86.22,86.22,86.22,1,3,3,3.0,...,0,0,0,0,0,0,0,0,0,0
0005e1862207bf6ccc02e4228effd9a0,26,1,150.12,150.12,150.12,150.12,1,4,4,4.0,...,0,0,0,0,0,0,0,0,0,0
00115fc7123b5310cf6d3a3aa932699e,68,1,76.11,76.11,76.11,76.11,1,4,4,4.0,...,0,0,0,0,0,0,0,0,0,1
001f3c4211216384d5fe59b041ce1461,11,1,35.84,35.84,35.84,35.84,1,3,3,3.0,...,0,0,0,0,0,0,0,0,0,0
002043098f10ba39a4600b6c52fbfe3c,4,1,237.81,237.81,237.81,237.81,1,4,4,4.0,...,0,0,0,0,0,0,0,0,0,0


In [6]:
data_6_months_fe = features_engineering(data_6_months)
data_6_months_fe.head()

,recency,frequency,monetary,payment_value_mean,payment_value_min,payment_value_max,review_score_count,review_score_min,review_score_max,review_score_mean,...,PCat_books_technical,PCat_dvds_blu_ray,PCat_security_and_services,PCat_tablets_printing_image,PCat_furniture_bedroom,PCat_cine_photo,PCat_fashio_female_clothing,PCat_books_imported,PCat_la_cuisine,payment_facility
customer_unique_id,,,,,,,,,,,,,,,,,,,,,
0000f46a3911fa3c0805444483337064,111,1,86.22,86.22,86.22,86.22,1,3,3,3.0,...,0,0,0,0,0,0,0,0,0,0
0005e1862207bf6ccc02e4228effd9a0,116,1,150.12,150.12,150.12,150.12,1,4,4,4.0,...,0,0,0,0,0,0,0,0,0,0
00115fc7123b5310cf6d3a3aa932699e,159,1,76.11,76.11,76.11,76.11,1,4,4,4.0,...,0,0,0,0,0,0,0,0,0,1
0011805441c0d1b68b48002f1d005526,66,1,297.14,297.14,297.14,297.14,1,3,3,3.0,...,0,0,0,0,0,0,0,0,0,0
0011857aff0e5871ce5eb429f21cdaf5,1,1,192.83,192.83,192.83,192.83,1,5,5,5.0,...,0,0,0,0,0,0,0,0,0,0


In [7]:
data_9_months_fe = features_engineering(data_9_months)
data_9_months_fe.head()

,recency,frequency,monetary,payment_value_mean,payment_value_min,payment_value_max,review_score_count,review_score_min,review_score_max,review_score_mean,...,PCat_dvds_blu_ray,PCat_cds_dvds_musicals,PCat_security_and_services,PCat_tablets_printing_image,PCat_furniture_bedroom,PCat_cine_photo,PCat_fashio_female_clothing,PCat_books_imported,PCat_la_cuisine,payment_facility
customer_unique_id,,,,,,,,,,,,,,,,,,,,,
0000f46a3911fa3c0805444483337064,203,1,86.22,86.22,86.22,86.22,1,3,3,3.0,...,0,0,0,0,0,0,0,0,0,0
0005e1862207bf6ccc02e4228effd9a0,209,1,150.12,150.12,150.12,150.12,1,4,4,4.0,...,0,0,0,0,0,0,0,0,0,0
0006fdc98a402fceb4eb0ee528f6a8d4,73,1,29.00,29.00,29.00,29.00,1,3,3,3.0,...,0,0,0,0,0,0,0,0,0,0
000a5ad9c4601d2bbdd9ed765d5213b3,49,1,91.28,91.28,91.28,91.28,1,4,4,4.0,...,0,0,0,0,0,0,0,0,0,0
000bfa1d2f1a41876493be685390d6d3,0,2,93.70,46.85,46.85,46.85,2,4,5,4.5,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
data_12_months_fe = features_engineering(data_12_months)
data_12_months_fe.head()

In [ ]:
data_all_fe = features_engineering(data_all)
data_all_fe.head()

## 3 - Normalisation

In [ ]:
standardize = preprocessing.StandardScaler(copy=True, with_mean=True, with_std=True)
standardize.fit(data_3_months_fe)
data_3_months_fe = standardize.transform(data_3_months_fe)

## 4 - Clustering à l'aide des baselines
### 4.1 - K-means

In [ ]:
base_km_cluster_number = []
base_km_score = []
for n in range(2, 25):
    kmean = KMeans(n_clusters=n, n_init=100, max_iter=3000)
    kmean.fit(data_3_months_fe)
    base_km_cluster_number.append(n)
    base_km_score.append(silhouette_score(data_3_months_fe, kmean.labels_, metric='euclidean'))

plt.figure(figsize=(10, 5))
plt.plot(base_km_cluster_number, base_km_score, color="skyblue", lw=2)
plt.xlabel("Nombre de cluster")
plt.ylabel("Score de silhouette")
plt.title("Baseline Kmean")
plt.tight_layout()
plt.show()


### 4.2 - DBSCAN

In [ ]:
dbscan_clust = DBSCAN(eps=0.8, min_samples=10)
dbscan_clust.fit(data_3_months_fe)

In [ ]:
dbscan_clust.labels_

In [ ]:
n_clusters = len(set(dbscan_clust.labels_)) - (1 if -1 in dbscan_clust.labels_ else 0)
n_clusters

In [ ]:
silhouette_score(data_3_months_fe, dbscan_clust.labels_)

In [ ]:
list(dbscan_clust.labels_).count(-1)

4148 Clients non présent dans un cluster

### 4.3 - Clustering Hierarchique

In [ ]:
base_hier_cluster_number = []
base_hier_score = []
for n in range(2, 25):
    clust_hier = AgglomerativeClustering(n_clusters=n)
    clust_hier.fit(data_3_months_fe)
    base_hier_cluster_number.append(n)
    base_hier_score.append(silhouette_score(data_3_months_fe, clust_hier.labels_, metric="euclidean"))

plt.figure(figsize=(10, 5))
plt.plot(base_hier_cluster_number, base_hier_score, color="skyblue", lw=2)
plt.xlabel("Nombre de cluster")
plt.ylabel("Score de silhouette")
plt.title("Baseline Clustering Hierarchique")
plt.tight_layout()
plt.show()

## 5- Réduction de dimension

In [ ]:
X = data_3_months_fe.values.compute()
dX = da.from_array(X, chunks=X.shape)

### 5.1 - Linear PCA
#### 5.1.1 - Decomposition et projection

In [ ]:
pca = PCA(n_components=94)
pca.fit(dX)

In [ ]:
components = range(1, len(pca.explained_variance_ratio_) + 1)
plt.figure(figsize=(10, 5))
plt.plot(components, pca.explained_variance_ratio_*100, color='skyblue', lw=2)
plt.xlabel('Nombre de composantes', fontsize=12)
plt.ylabel('Pourcentage Variance Expliquée', fontsize=12)
plt.title('PCA - part de variance expliquée par composantes')
plt.tight_layout()
plt.show()

In [ ]:
pca = PCA(n_components=10)
pca.fit(dX)

X_pca_projected = pca.transform(dX)

#### 5.1.2 - Clustering
##### 5.1.2.1 - K-means

In [ ]:
pca_km_cluster_number = []
pca_km_score = []
for n in range(2, 25):
    kmean = KMeans(n_clusters=n, n_init=100, max_iter=3000)
    kmean.fit(X_pca_projected)
    pca_km_cluster_number.append(n)
    pca_km_score.append(silhouette_score(X_pca_projected, kmean.labels_, metric='euclidean'))

plt.figure(figsize=(10, 5))
plt.plot(pca_km_cluster_number, pca_km_score, color="skyblue", lw=2)
plt.xlabel("Nombre de cluster")
plt.ylabel("Score de silhouette")
plt.title("PCA Kmean")
plt.tight_layout()
plt.show()


##### 5.1.2.2 - Clustering Hierarchique

In [ ]:
pca_hier_cluster_number = []
pca_hier_score = []
for n in range(2, 25):
    clust_hier = AgglomerativeClustering(n_clusters=n)
    clust_hier.fit(X_pca_projected)
    pca_hier_cluster_number.append(n)
    pca_hier_score.append(silhouette_score(X_pca_projected, clust_hier.labels_, metric="euclidean"))

plt.figure(figsize=(10, 5))
plt.plot(pca_hier_cluster_number, pca_hier_score, color="skyblue", lw=2)
plt.xlabel("Nombre de cluster")
plt.ylabel("Score de silhouette")
plt.title("PCA Clustering Hierarchique")
plt.tight_layout()
plt.show()

### 5.2 -  Kernel PCA
#### 5.2.1 - Decomposition et projection

In [ ]:
kpca = decomposition.KernelPCA(n_components=101, kernel="cosine")
kpca.fit(dX)

In [ ]:
components = range(1, len(kpca.lambdas_)+1)
plt.figure(figsize=(10, 5))
plt.plot(components, kpca.lambdas_*100, color='skyblue', lw=2)
plt.xlabel('Nombre de composantes', fontsize=12)
plt.ylabel('AgainValues', fontsize=12)
plt.title('KernelPCA - part de variance expliquée par composantes')
plt.tight_layout()
plt.show()

In [ ]:
kpca = decomposition.KernelPCA(n_components=10, kernel="cosine")
kpca.fit(dX)

X_kpca_projected = kpca.transform(dX)

#### 5.2.2 - Clustering
##### 5.2.2.1 - K-means

In [ ]:
kpca_km_cluster_number = []
kpca_km_score = []
for n in range(2, 25):
    kmean = KMeans(n_clusters=n, n_init=100, max_iter=3000)
    kmean.fit(X_kpca_projected)
    kpca_km_cluster_number.append(n)
    kpca_km_score.append(silhouette_score(X_kpca_projected, kmean.labels_, metric='euclidean'))

plt.figure(figsize=(10, 5))
plt.plot(kpca_km_cluster_number, kpca_km_score, color="skyblue", lw=2)
plt.xlabel("Nombre de cluster")
plt.ylabel("Score de silhouette")
plt.title("KernelPCA Kmean")
plt.tight_layout()
plt.show()

##### 5.2.2.2 - Clustering Hierarchique

In [ ]:
kpca_hier_cluster_number = []
kpca_hier_score = []
for n in range(2, 25):
    clust_hier = AgglomerativeClustering(n_clusters=n)
    clust_hier.fit(X_kpca_projected)
    kpca_hier_cluster_number.append(n)
    kpca_hier_score.append(silhouette_score(X_kpca_projected, clust_hier.labels_, metric="euclidean"))

plt.figure(figsize=(10, 5))
plt.plot(kpca_hier_cluster_number, kpca_hier_score, color="skyblue", lw=2)
plt.xlabel("Nombre de cluster")
plt.ylabel("Score de silhouette")
plt.title("KernelPCA Clustering Hierarchique")
plt.tight_layout()
plt.show()

### 5.3 - T-SNE
#### 5.3.1 - Decomposition et projection

In [ ]:
tsne = manifold.TSNE(n_components=2, perplexity=50, n_iter=3000, random_state=0)
X_tsne = tsne.fit_transform(dX)

In [ ]:
x = []
y = []

for i in range(X_tsne.shape[0]):
    x.append(X_tsne[i, 0])
    y.append(X_tsne[i, 1])

plt.figure(figsize=(10, 5))
plt.scatter(x, y)
plt.tight_layout()
plt.show()

#### 5.3.2 - Clustering
##### 5.3.2.1 - K-means

In [ ]:
tsne_km_cluster_number = []
tsne_km_score = []
for n in range(2, 25):
    kmean = KMeans(n_clusters=n, n_init=100, max_iter=3000)
    kmean.fit(X_tsne)
    tsne_km_cluster_number.append(n)
    tsne_km_score.append(silhouette_score(X_tsne, kmean.labels_, metric='euclidean'))

plt.figure(figsize=(10, 5))
plt.plot(tsne_km_cluster_number, tsne_km_score, color="skyblue", lw=2)
plt.xlabel("Nombre de cluster")
plt.ylabel("Score de silhouette")
plt.title("TSNE Kmean")
plt.tight_layout()
plt.show()

##### 5.3.2.2 - Clustering Hierarchique

In [ ]:
tsne_hier_cluster_number = []
tsne_hier_score = []
for n in range(2, 25):
    clust_hier = AgglomerativeClustering(n_clusters=n)
    clust_hier.fit(X_tsne)
    tsne_hier_cluster_number.append(n)
    tsne_hier_score.append(silhouette_score(X_tsne, clust_hier.labels_, metric="euclidean"))

plt.figure(figsize=(10, 5))
plt.plot(tsne_hier_cluster_number, tsne_hier_score, color="skyblue", lw=2)
plt.xlabel("Nombre de cluster")
plt.ylabel("Score de silhouette")
plt.title("TSNE Clustering Hierarchique")
plt.tight_layout()
plt.show()

### 5.4 - MDS
#### 5.4.1 - Decomposition et projection

In [ ]:
mds = manifold.MDS(n_components=2, n_jobs=-1)
X_mds = mds.fit_transform(dX)

#### 5.4.2 - Clustering
##### 5.4.2.1 - K-means

In [ ]:
mds_km_cluster_number = []
mds_km_score = []
for n in range(2, 25):
    kmean = KMeans(n_clusters=n, n_init=100, max_iter=3000)
    kmean.fit(X_mds)
    mds_km_cluster_number.append(n)
    mds_km_score.append(silhouette_score(X_mds, kmean.labels_, metric='euclidean'))

plt.figure(figsize=(10, 5))
plt.plot(mds_km_cluster_number, mds_km_score, color="skyblue", lw=2)
plt.xlabel("Nombre de cluster")
plt.ylabel("Score de silhouette")
plt.title("MDS Kmean")
plt.tight_layout()
plt.show()

##### 5.4.2.2 - Clustering Hierarchique

In [ ]:
mds_hier_cluster_number = []
mds_hier_score = []
for n in range(2, 25):
    clust_hier = AgglomerativeClustering(n_clusters=n)
    clust_hier.fit(X_mds)
    mds_hier_cluster_number.append(n)
    mds_hier_score.append(silhouette_score(X_mds, clust_hier.labels_, metric="euclidean"))

plt.figure(figsize=(10, 5))
plt.plot(mds_hier_cluster_number, mds_hier_score, color="skyblue", lw=2)
plt.xlabel("Nombre de cluster")
plt.ylabel("Score de silhouette")
plt.title("MDS Clustering Hierarchique")
plt.tight_layout()
plt.show()

### 5.5 - ISO Map
#### 5.5.1 - Decomposition et projection

In [ ]:
isomap = manifold.Isomap(n_components=2, n_jobs=-1)
X_isomap = isomap.fit_transform(dX)

#### 5.5.2 - Clustering
##### 5.5.2.1 - K-means

In [ ]:
iso_km_cluster_number = []
iso_km_score = []
for n in range(2, 25):
    kmean = KMeans(n_clusters=n, n_init=100, max_iter=3000)
    kmean.fit(X_isomap)
    iso_km_cluster_number.append(n)
    iso_km_score.append(silhouette_score(X_isomap, kmean.labels_, metric='euclidean'))

plt.figure(figsize=(10, 5))
plt.plot(iso_km_cluster_number, iso_km_score, color="skyblue", lw=2)
plt.xlabel("Nombre de cluster")
plt.ylabel("Score de silhouette")
plt.title("ISOMap Kmean")
plt.tight_layout()
plt.show()

##### 5.5.2.2 - Clustering Hierarchique

In [ ]:
iso_hier_cluster_number = []
iso_hier_score = []
for n in range(2, 25):
    clust_hier = AgglomerativeClustering(n_clusters=n)
    clust_hier.fit(X_isomap)
    iso_hier_cluster_number.append(n)
    iso_hier_score.append(silhouette_score(X_isomap, clust_hier.labels_, metric="euclidean"))

plt.figure(figsize=(10, 5))
plt.plot(iso_hier_cluster_number, iso_hier_score, color="skyblue", lw=2)
plt.xlabel("Nombre de cluster")
plt.ylabel("Score de silhouette")
plt.title("ISOMap Clustering Hierarchique")
plt.tight_layout()
plt.show()